# Predict Student Performance

In [1]:
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

2025-02-22 00:37:59.464208: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-22 00:38:00.119176: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-02-22 00:38:00.119208: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-02-22 00:38:00.221584: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 00:38:02.095218: W tensorflow/stream_executor/platform/de

## Set variable

In [2]:
PIPELINE_NAME = "student-performance-pipeline"
SCHEMA_PIPELINE_NAME = "student-performance-tfdv-schema"

# Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

DATA_ROOT = "data"

In [3]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Pengolahan Data

### Data Ingestion

In [4]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [5]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [6]:
interactive_context.show(example_gen.outputs['examples'])

### Data Validation

#### Summary Statistic

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

#### Data Schema

In [9]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Attendance (%)',FLOAT,required,,-
'Grades',FLOAT,required,,-
'Sleep Hours',FLOAT,required,,-
'Socioeconomic Score',FLOAT,required,,-
'Study Hours',FLOAT,required,,-


#### Anomali pada dataset

In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [12]:
interactive_context.show(example_validator.outputs['anomalies'])

### Data Preprocessing

In [13]:
TRANSFORM_MODULE_FILE = "student_performance_transform.py"

In [14]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
    
def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    outputs = {}

    attendance = inputs['Attendance (%)']
    outputs['normalized_attendance'] = tft.scale_to_z_score(attendance)

    sleep_hours = inputs['Sleep Hours']
    outputs['normalized_sleep_hours'] = tft.scale_to_0_1(sleep_hours)

    socioeconomic_score = inputs['Socioeconomic Score']
    outputs['normalized_socioeconomic_score'] = tft.scale_to_0_1(socioeconomic_score)

    study_hours = inputs['Study Hours']
    outputs['normalized_study_hours'] = tft.scale_to_0_1(study_hours)
    
    grades = inputs['Grades']
    outputs['normalized_grades'] = tft.scale_to_0_1(grades)

    return outputs

Overwriting student_performance_transform.py


In [15]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying student_performance_trainer.py -> build/lib
copying student_performance_tuner.py -> build/lib
copying student_performance_transform.py -> build/lib
installing to /tmp/tmphftgxctl
running install
running install_lib
copying build/lib/student_performance_trainer.py -> /tmp/tmphftgxctl
copying build/lib/student_performance_tuner.py -> /tmp/tmphftgxctl
copying build/lib/student_performance_transform.py -> /tmp/tmphftgxctl
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/S

/home/ardi/miniconda3/envs/ml-predict-student-performance/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


adding 'student_performance_trainer.py'
adding 'student_performance_transform.py'
adding 'student_performance_tuner.py'
adding 'tfx_user_code_Transform-0.0+53b4f0344adb37df71ff5ceaaa868710316104780929106e4f3b64bd33462b3a.dist-info/METADATA'
adding 'tfx_user_code_Transform-0.0+53b4f0344adb37df71ff5ceaaa868710316104780929106e4f3b64bd33462b3a.dist-info/WHEEL'
adding 'tfx_user_code_Transform-0.0+53b4f0344adb37df71ff5ceaaa868710316104780929106e4f3b64bd33462b3a.dist-info/top_level.txt'
adding 'tfx_user_code_Transform-0.0+53b4f0344adb37df71ff5ceaaa868710316104780929106e4f3b64bd33462b3a.dist-info/RECORD'
removing /tmp/tmphftgxctl
Processing ./pipelines/student-performance-pipeline/_wheels/tfx_user_code_Transform-0.0+53b4f0344adb37df71ff5ceaaa868710316104780929106e4f3b64bd33462b3a-py3-none-any.whl
Processing ./pipelines/student-performance-pipeline/_wheels/tfx_user_code_Transform-0.0+53b4f0344adb37df71ff5ceaaa868710316104780929106e4f3b64bd33462b3a-py3-none-any.whl
Processing ./pipelines/student

2025-02-22 00:38:23.052861: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-02-22 00:38:23.053150: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2025-02-22 00:38:23.053203: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ardi-device): /proc/driver/nvidia/version does not exist
2025-02-22 00:38:23.053530: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/e555f6ded48b418fa1225dce7d5b0bfa/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Pengembangan dan Validasi Model

### Tuner Model

In [16]:
TUNER_MODULE_FILE = "student_performance_tuner.py"

In [17]:
%%writefile {TUNER_MODULE_FILE}

import keras_tuner as kt
import tensorflow as tf
import tensorflow_transform as tft
from tfx.components.tuner.component import TunerFnResult

NUM_EPOCHS = 10
LABEL_KEY="normalized_grades"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=32) -> tf.data.Dataset:
    # Get post_transform feature spec
    transform_feature_spec = (tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=LABEL_KEY
    )

    return dataset

def build_model(hp, feature_spec):
    inputs = {feature: tf.keras.layers.Input(shape=(1,), name=feature) for feature in feature_spec.keys() if feature != LABEL_KEY}
    concatenated = tf.keras.layers.Concatenate()(list(inputs.values()))

    x = concatenated
    for i in range(hp.Int("num_layers", 1, 3)):
        x = tf.keras.layers.Dense(
            units=hp.Int(f"units_{i}", min_value=32, max_value=128, step=32),
            activation="relu"
        )(x)

    outputs = tf.keras.layers.Dense(1, activation="linear")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])
        ),
        loss="mse",
        metrics=["mae"]
    )
    
    return model

# Fungsi tuner untuk pipeline TFX
def tuner_fn(fn_args):
    # Load data dari transform
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    feature_spec = tf_transform_output.transformed_feature_spec()

    train_dataset = input_fn(fn_args.train_files, tf_transform_output, NUM_EPOCHS, batch_size=32)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, NUM_EPOCHS, batch_size=32)

    tuner = kt.Hyperband(
        hypermodel=lambda hp: build_model(hp, feature_spec),
        objective='val_mae',
        max_epochs=NUM_EPOCHS,
        factor=3,
        executions_per_trial=1,
        directory=fn_args.working_dir,
        project_name='student_performance_tuning'
    )
    
    tuner.search(
        train_dataset,
        validation_data=eval_dataset,
        epochs=NUM_EPOCHS,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
    )
        
    return TunerFnResult(
        tuner=tuner, 
        fit_kwargs={
            'x': train_dataset,
            'validation_data': eval_dataset,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps
    })

Overwriting student_performance_tuner.py


In [18]:
from tfx.proto import trainer_pb2

tuner = Tuner(
    module_file=os.path.abspath('student_performance_tuner.py'),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=500),
)

interactive_context.run(tuner)

Trial 42 Complete [00h 00m 02s]
val_mae: 0.050269559025764465

Best val_mae So Far: 0.012509205378592014
Total elapsed time: 00h 03m 00s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines/student-performance-pipeline/.temp/6/student_performance_tuning
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 2
units_0: 64
learning_rate: 0.01
units_1: 96
units_2: 128
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.012509205378592014
Trial summary
Hyperparameters:
num_layers: 3
units_0: 96
learning_rate: 0.01
units_1: 96
units_2: 32
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
Score: 0.0135134756565094
Trial summary
Hyperparameters:
num_layers: 3
units_0: 128
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
units_1: 32
units_2: 32
Score: 0.013579954393208027
Trial summary
Hyperparameters:
num_layers: 3
units_0: 96
learning_rate: 0.01
units_1: 32
units_2: 96
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0015
Score: 0.013905826956033707
Trial summary
Hyperparameters:


ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

### Training Model

In [19]:
TRAINER_MODULE_FILE="student_performance_trainer.py"

In [20]:
%%writefile {TRAINER_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_transform.beam.impl as tft_beam
from tfx.components.trainer.fn_args_utils import FnArgs
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx_bsl.public import tfxio
from tfx.examples.penguin import penguin_utils_base as base

# Feature keys
FEATURE_KEYS = [
    'normalized_attendance',
    'normalized_sleep_hours',
    'normalized_socioeconomic_score',
    'normalized_study_hours'
]

LABEL_KEY = 'normalized_grades'

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=32) -> tf.data.Dataset:
    # Get post_transform feature spec
    transform_feature_spec = (tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=LABEL_KEY)

    return dataset

def build_model(hp, tf_transform_output):
    """Membangun model dengan hyperparameters dari Tuner."""

    feature_spec = tf_transform_output.transformed_feature_spec().copy()

    inputs = {feature: tf.keras.layers.Input(shape=(1,), name=feature) for feature in feature_spec.keys() if feature != LABEL_KEY}
    concatenated = tf.keras.layers.Concatenate()(list(inputs.values()))  # Gabungkan semua fitur

    x = concatenated
    for i in range(hp["num_layers"]):
        x = tf.keras.layers.Dense(
            units=hp[f"units_{i}"],
            activation="relu"
        )(x)

    outputs = tf.keras.layers.Dense(1, activation="linear")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Hyperparameter: Learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp["learning_rate"]
        ),
        loss="mse",
        metrics=["mae"]
    )
    
    return model
    
def run_fn(fn_args: FnArgs):
    """Fungsi utama Trainer untuk melatih model."""
    

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Membangun model
    model = build_model(fn_args.hyperparameters["values"], tf_transform_output)

    # Membaca dataset
    train_dataset = input_fn(fn_args.train_files, tf_transform_output, 10)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, 10)

    # Callback untuk menyimpan model terbaik
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        save_best_only=True
    )

    # Melatih model
    model.fit(
        train_dataset,
        validation_data=eval_dataset,
        epochs=10,
        callbacks=[model_checkpoint]
    )

    # Menyimpan model akhir
    model.save(fn_args.serving_model_dir, save_format='tf')

Overwriting student_performance_trainer.py


In [21]:
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']),
)

interactive_context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying student_performance_trainer.py -> build/lib
copying student_performance_tuner.py -> build/lib
copying student_performance_transform.py -> build/lib
installing to /tmp/tmp2wkrxmq7
running install
running install_lib
copying build/lib/student_performance_trainer.py -> /tmp/tmp2wkrxmq7
copying build/lib/student_performance_tuner.py -> /tmp/tmp2wkrxmq7
copying build/lib/student_performance_transform.py -> /tmp/tmp2wkrxmq7
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Co

/home/ardi/miniconda3/envs/ml-predict-student-performance/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipelines/student-performance-pipeline/_wheels/tfx_user_code_Trainer-0.0+f2fea7fb547d4b64398761dd6e4944142229184772a209cc476af62e804fc967-py3-none-any.whl
Epoch 1/10
    338/Unknown - 2s 3ms/step - loss: 0.0042 - mae: 0.0408INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/7/Format-Serving/assets


344/344 [==============================] - 3s 8ms/step - loss: 0.0041 - mae: 0.0406 - val_loss: 0.0012 - val_mae: 0.0253
Epoch 2/10
323/344 [===========================>..] - ETA: 0s - loss: 7.2076e-04 - mae: 0.0190INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/7/Format-Serving/assets


...

344/344 [==============================] - 3s 8ms/step - loss: 7.0024e-04 - mae: 0.0188 - val_loss: 3.9330e-04 - val_mae: 0.0150
Epoch 3/10
344/344 [==============================] - 2s 5ms/step - loss: 5.2271e-04 - mae: 0.0166 - val_loss: 7.9704e-04 - val_mae: 0.0188
Epoch 4/10
344/344 [==============================] - 1s 3ms/step - loss: 4.1138e-04 - mae: 0.0151 - val_loss: 4.1677e-04 - val_mae: 0.0151
Epoch 5/10
344/344 [==============================] - 1s 4ms/step - loss: 4.2074e-04 - mae: 0.0151 - val_loss: 8.6134e-04 - val_mae: 0.0219
Epoch 6/10
344/344 [==============================] - 1s 3ms/step - loss: 3.6482e-04 - mae: 0.0145 - val_loss: 4.9848e-04 - val_mae: 0.0169
Epoch 7/10
336/344 [============================>.] - ETA: 0s - loss: 4.7300e-04 - mae: 0.0158INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/7/Format-Serving/assets


344/344 [==============================] - 2s 7ms/step - loss: 4.7341e-04 - mae: 0.0159 - val_loss: 3.2752e-04 - val_mae: 0.0139
Epoch 8/10
344/344 [==============================] - 1s 3ms/step - loss: 4.7820e-04 - mae: 0.0157 - val_loss: 7.5201e-04 - val_mae: 0.0195
Epoch 9/10
344/344 [==============================] - 1s 3ms/step - loss: 3.8482e-04 - mae: 0.0146 - val_loss: 4.0694e-04 - val_mae: 0.0152
Epoch 10/10
344/344 [==============================] - 1s 3ms/step - loss: 4.5643e-04 - mae: 0.0157 - val_loss: 4.4314e-04 - val_mae: 0.0154
INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/7/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

### Analisis dan Validasi Model

#### Komponen Resolver

In [22]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

#### Komponen Evaluator

In [23]:
import tensorflow_model_analysis as tfma 
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='normalized_grades')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5
                    }),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
)

In [24]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [25]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'mae': {'dou…

## Model Deployment

In [26]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory='serving_model_dir/predict-student-performance-model'))
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

In [27]:
interactive_context.show(pusher)

Pusher(spec: <tfx.types.standard_component_specs.PusherSpec object at 0x75ba1b31f5b0>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x75ba1995d310>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: Pusher, inputs: {'model': OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}), 'model_blessing': OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={})}, outputs: {'pushed_model': OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={})})

## Test Model

In [30]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/predict-student-performance-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1740156135'}]}
